# Hybrid Movie Recommendation System
Authors of the inspiring paper: Poonam Sharma, Lokesh Yadav

Author: Nikolaus Czernin

## Contents
1. Preparations
1. Ratings Data Preprocessing (user-item-rating-matrix)
2. Keywords Data Preprocessing (item-keywords-binary-matrix)
3. Content-based filtering algorithm
6. Item-based collaborative filtering algorithm
7. Hybrid RecSys algorithm
8. Testing
10. Evaluation
11. References


## 1. Preparations

In [18]:
import pandas as pd
import numpy as np
import json
import ast
from pandas.api.types import CategoricalDtype
import sys
from IPython.display import clear_output
import time
import csv
from math import floor, ceil

from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

### Setting a limit on the datasize
This is necessary to compute fast results. 
The algorithms remain the same, it's just a bit faster.
Feel free to set the limits as you like.
Setting the limits to 200 will take 1–2 minutes to run.

In [19]:
# set a limit on how many users and movies you want to include
usersLimit = 200
itemLimit = 200

### Sparse Pivot Tables
The whole dataset with all its NAs (e.g. a user not having rated a movie yet) would not even fit into my RAM. The Sparse Matrix format from SciPy allows creating huge matrices with a lot of missing values without having to reserve as much memory. 

In [20]:
# sparse pivot table creator
def get_sparse_pivot_table(df, rowCol, colCol, valCol):
    """
    Returns a sparse pivot table requiring very little memory.
    df: A dataframe with at least the following three columns: rowCol, colCol, valCol
    rowCol values will be the indices of the rows
    colCol values will be the indices of the columns
    valCol[i] values will be the value of the resulting dataframes value [rowCol[i], valCol[i]]
    NAs are not acceptable
    """
    # create factor arrays for the rows and columns
    rowCol_cat = CategoricalDtype(sorted(df[rowCol].unique()), ordered=True)
    colCol_cat = CategoricalDtype(sorted(df[colCol].unique()), ordered=True)

    # create the rows and columns from the respective factors
    row = df[rowCol].astype(rowCol_cat).cat.codes
    col = df[colCol].astype(colCol_cat).cat.codes

    # create the sparse matrix
    # 1: pass the values for the cells array
    # 2: pass the indices (row) and the names (col)
    # 3: pass the dimensions (length of row and length of col)
    sparse_matrix = csr_matrix((df[valCol], (row, col)), shape=(rowCol_cat.categories.size, colCol_cat.categories.size))
    # turn it into a sparse Pandas Dataframe
    dfs = pd.DataFrame.sparse.from_spmatrix(sparse_matrix)
    #return dfs
    return sparse_matrix

# turning a sprase pivot table into a pd.dataframe (optional)
def get_sparse_pandas_df(sparse_matrix):
    """
    Returns a Pandas Dataframe
    sparse_matrix: a Scipy.Sparse Matrix (e.g. created from get_sparse_pivot_table()) 
    """
    return pd.DataFrame.sparse.from_spmatrix(sparse_matrix)

## 2. Ratings Data Preprocessing (user-item-rating-matrix)
This is a dataset with the following columns:
- `userId`: int
- `movieId`: int, this will later be referred to be the "itemId"
- `rating`: float, can be anything between 0.5 and 5 (steps of 0.5), a rating of 0 means a movie hasn't been rated by that user
- `timestamp`: int, not relevant for this project

It will be turned into a Matrix with ...
- `userId`, i.e. one user, per row
- `movieId`, i.e. one movie/item, per column
- `rating`, i.e. the rating of one user given to one item, as values

In [21]:
# read the data from the csv
userItemRatings = pd.read_csv("ratings_small.csv")
userItemRatings = userItemRatings[userItemRatings['userId']<usersLimit]
userItemRatings = userItemRatings[userItemRatings['movieId']<itemLimit]

# call the Sparse Pivot Table Function
userItemRatings = get_sparse_pivot_table(userItemRatings, "userId", "movieId", "rating").astype(int)
#display(userItemRatings.todense())
print(userItemRatings.shape)
print("Filesize of the matrix:", sys.getsizeof(userItemRatings)) # this is significantly smaller than creating it entirely with Pandas Pivot Tables

(182, 149)
Filesize of the matrix: 48


In [22]:
# by calling get_sparse_pandas_df(userItemRatings), you get a pandas dataframe
if itemLimit < 1000:
    r = get_sparse_pandas_df(userItemRatings)
    display(r)

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,4,...,3,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
4,0,0,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
178,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 3. Keywords Data Preprocessing (item-keywords-binary-matrix)
This is a dataset with the following columns:
- `id`: int, the itemId/movieId
- `keywords`: json-string, each object in this array is a keyword, containing the `id` (int) and the correpsonding `name` (str)


It will be turned into a Matrix with ...
- `movieId`, i.e. one movie/item, per row
- `keyword`, i.e. one keyword, per column
- `match`, i.e. whether that item/movie is associated with that particular keyword

In [23]:
# Load the Keywords Dataset
# 1st col: movieId
# 2nd col: array of dictionaries (2 key-item pairs: keyword id, keyword name) as string
itemKeywordList = pd.read_csv(
    "keywords.csv", 
    engine="python", 
    error_bad_lines=False
)

itemKeywordList = itemKeywordList[itemKeywordList['id']<itemLimit]
# turn the strings into python lists
# cannot use JSON functions, as JSON format rules are not met (single quotes instead of double quotes)
# literal_eval takes the literal meaning of the string as Python code, which results in valid lists
itemKeywordList.keywords = itemKeywordList.keywords.apply(lambda x: ast.literal_eval(x))

# rather than having one row per movie with an array of keyword ids
# … we want a df with one row per keyword per movie
itemsKeywordsLonger = {}
j = 0
for i, row in itemKeywordList.iterrows():
    for keyword in row.keywords:
        itemsKeywordsLonger[j] = {'movieId': row.id, 'keyword': keyword['id']}
        j += 1
# rows and columns are swapped, transpose it
itemsKeywordsLonger = pd.DataFrame(itemsKeywordsLonger).transpose()
# remove duplicates
itemsKeywordsLonger = itemsKeywordsLonger.drop_duplicates()


# not every movieId as keyword data
# we have to create an empty movie_id-keyword line for every movie without keyword data
# the function creating the sparse numpy arrays will otherwise set the smallest given movieId (in this case 5) to be 0
# get the movieIds that are represented in the itemKeywordsMatrix data
givenItemIds = itemsKeywordsLonger.movieId.unique()

# loop through all movieIds we have
for i in range(itemLimit):
    # if the id does not have data
    if i not in givenItemIds:
        # add an empty row to the data
        # we cannot set the keyword value to NaN, as the get_sparse_pivot_table() function would throw an error
        # instead we set it to 0, thus creating a column for a non-existing keyword which matches with every movie not having any keyword
        # we can then delete that column after the sparse matrix was created
        newRow = {'movieId': i, 'keyword': 0}
        itemsKeywordsLonger = itemsKeywordsLonger.append(newRow, ignore_index = True)


# create a column with the value 1; this will be the value for every match in the pivot table
itemsKeywordsLonger["match"] = 1
display(itemsKeywordsLonger)

# call the Sparse Pivot Table Function
itemKeywordsMatrix = get_sparse_pivot_table(itemsKeywordsLonger, "movieId", "keyword", "match")

# delete the first column
itemKeywordsMatrix = itemKeywordsMatrix[:,1:]

print("Filesize of the matrix:", sys.getsizeof(itemKeywordsMatrix))
print("Shape of the matrix:", itemKeywordsMatrix.shape)

Skipping line 4235: unexpected end of data


,movieId,keyword,match
0,5,612,1
1,5,613,1
2,5,616,1
3,5,622,1
4,5,922,1
...,...,...,...
1196,188,0,1
1197,189,0,1
1198,190,0,1
1199,191,0,1


Filesize of the matrix: 48
Shape of the matrix: (200, 855)


In [24]:
# by calling get_sparse_pandas_df(itemKeywordsMatrix), you get a pandas dataframe
if itemLimit < 1000:
    mk = get_sparse_pandas_df(itemKeywordsMatrix)
    display(mk)

,0,1,2,3,4,5,6,7,8,9,...,845,846,847,848,849,850,851,852,853,854
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
196,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
197,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
198,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


### Make a dataframe with all keywords (optional)
You can use this to check for the meanings of the keywords associated with an item of interest.

In [25]:
# make a df with all keywords (optional)
# use list comprehension to gather all keyword dictionaries
keywordsList = [keyword for row in itemKeywordList.keywords for keyword in row]
# make a Pandas DF from it
keywordsDf = pd.DataFrame(keywordsList)
keywordsDf = keywordsDf.drop_duplicates()
keywordsDf = keywordsDf.set_index('id')
keywordsDf = keywordsDf.sort_index()
keywordsDf

,name
id,
30,individual
74,germany
83,saving the world
90,paris
107,barcelona spain
...,...
235792,2nd century
235793,successor
236316,anarchic comedy


## 4. Content-based filtering algorithm
Content-based filtering methods analyses the items a user likes for its contextual attributes/keywords and finds statistically similar items. If an item is very similar to an item the user likes, he may like that item as well. 

1. The proposed algorithm creates a user profile, which determines, which item-keywords a user tends to like. To be clear, when a user rates an item associated with the keywords "action", "gunslinger" and "western" highly (3 or higher of 5), those keywords will be added to the user profile. 

2. The algorithm then uses the rows of the `itemKeywordsMatrix` as item-profiles. 

3. It then calculates the similarity between all item-profiles and the user-profile. The top most similar items are viable recommendations. 



In [26]:
# creating user-profiles from their likes
def get_user_keyword_profile(userItemRatings, userId, itemKeywordsMatrix, likingThreshold = 3):
    """
    Input: 
    userItemRatings: 1-5 int user-ratings-matrix (sparse, 0 if unrated)
    userId: int of user you want the profile of
    itemKeywordsMatrix: binary movie-keyword-matrix (sparse)
    Output: 
    userProfile: binary array for user liking a keyword
    """
    userRatings = userItemRatings[userId]

    # turn the ordinal ratings into binary to indicate liking a movie
    # this requires a dense array, which is not problematic, as we are only looking at one user profile
    movieLikes = np.array(userRatings.todense())
    movieLikes = np.where(movieLikes >= likingThreshold)[1]

    # get a list of ids of the movies the user liked
    movieLikeIds = [likeId for likeId, like in enumerate(movieLikes) if like]    
    # select the liked ids from the movies*keywords matrix
    likedMoviesKeywords = itemKeywordsMatrix[movieLikeIds]
    # now we have a matrix with one row for each liked movie and a binary match for each keyword
    # sum the matches vertically 
    userProfile = likedMoviesKeywords.sum(axis=0)

    # now reduce the numbers to be either 1 for liking a keyword or else 0
    userProfile = userProfile
    userProfile = (userProfile>0).astype(int)

    #print("User Profile created")
    return userProfile

   


In [27]:
# function for comparing a profile with all movies' profiles to get similarities
def get_keyword_profile_similarity(profile_1, profile_2):
    """
    Returns the cosine similarity of two profiles
    """
    # loop through all movies and calculate cosine similarity to userProfile
    sim = cosine_similarity(profile_1, profile_2)
    return sim

# function for comparing a user-profile with all movie profiles to get similarities
def get_user_movie_keyword_similarity(userProfile, itemKeywordsMatrix):
    """
    Returns the cosine similarity of a user profile and and all item profiles
    """
    # reshape the user profile to be a 1d matrix (necessary to get cosine similarity)
    userProfile = np.array(userProfile)
    # calculate the similarity between the user profile and all item profiles
    sim = get_keyword_profile_similarity(userProfile, itemKeywordsMatrix)
    return sim

# Example
user = get_user_keyword_profile(userItemRatings, 69, itemKeywordsMatrix)
get_user_movie_keyword_similarity(user, itemKeywordsMatrix)

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.]])

In [28]:
%%time

# perform content based filtering to get n recommendations for a user
def perform_cbf(userItemRatings, userId, itemKeywordsMatrix, n=10):
    """
    Returns top n movies most similar to the given user (itemId, similarity_to_the_user_profile)
    userItemRatings: matrix with users in rows, items in columns and ratings as values
    userId: int, 
    itemKeywordsMatrix: matrix with items in rows, keywords in columns and matches as values
    """
    # get a userProfile(binary array. 1 indicates the user likes the keyword)
    userProfile = get_user_keyword_profile(userItemRatings, userId, itemKeywordsMatrix)
    # get the similarities between each movie and the user profile
    simRowCos = get_keyword_profile_similarity(userProfile, itemKeywordsMatrix)
    # get the ids of the top similarities
    topSimilarMovieIds = np.argsort(simRowCos)[0][::-1][:n]
    topSims = simRowCos[0][topSimilarMovieIds]
    return [(id, simRowCos[0][id]) for id in topSimilarMovieIds]
  
# Example
userId = 1
contentBasedFilteringRec = perform_cbf(userItemRatings, userId, itemKeywordsMatrix)
print("Content-based recommendations for user", userId)
contentBasedFilteringRec

Content-based recommendations for user 1
CPU times: user 4.01 ms, sys: 123 µs, total: 4.13 ms
Wall time: 3.44 ms


[(14, 0.5091750772173154),
 (13, 0.4843221048378523),
 (11, 0.4444444444444446),
 (15, 0.35136418446315326),
 (5, 0.35136418446315326),
 (6, 0.2484519974999766),
 (115, 0.07647191129018724),
 (114, 0.07027283689263066),
 (103, 0.0670025210172808),
 (18, 0.06537204504606135)]

In [29]:
def predict_rating_cbf(userId, itemId, userItemRatings, itemKeywordsMatrix):
    """
    Content-based filtering
    userId, itemId: int, the user-id and item-id that you want to predict a rating for
    userItemRatings: matrix with users in rows, items in columns and ratings as values
    itemKeywordsMatrix: matrix with items in rows, keywords in columns and matches as values
    """
    # select the user's row
    userRow = userItemRatings.todense()[userId]
    userRow = np.array(userRow)[0]
    # get the ids of the rated and unrated movies
    ratedIds = userRow != 0

    # get the unrated film's keyword profile
    unratedFilmProfile = itemKeywordsMatrix[itemId]
    
    # get similarities to all films he rated
    sim = get_keyword_profile_similarity(unratedFilmProfile, itemKeywordsMatrix[ratedIds])
    sim = sim[0]
    
    # normalize the similarities to add up to 1
    sim = sim / sum(sim)
    # multiply each normalized similarity with the rating of the movie you compared the unrated movie to
    weightedRatings = userRow[ratedIds] * sim
    # sum up the weighted ratings and round the result
    # this is the rating as predicted with content-based filtering
    predictedRating = sum(weightedRatings)
    # round the predicted rating to be int
    # if the sum is np.nan, you cannot round it, just put it to 0
    if np.isnan(predictedRating):
        predictedRating = 0
    predictedRating = round(predictedRating)
    # return the predicted rating
    return predictedRating

### Predicting item ratings using content-based filtering

In [30]:
# Example
for userId in range(3):
    for itemId in range(100):
        try:
            rating = predict_rating_cbf(userId, itemId, userItemRatings, itemKeywordsMatrix)
            if rating > 1 or (userId == 0 and itemId < 2):
                print(f"Predicted Rating for movie: {itemId} and user {userId}:", rating)
        except Exception as e:
            print(e)
            #raise e
            continue

<ipython-input-29-a44bce28ccea>:22: RuntimeWarning: invalid value encountered in true_divide
  sim = sim / sum(sim)


Predicted Rating for movie: 0 and user 0: 0
Predicted Rating for movie: 1 and user 0: 0
Predicted Rating for movie: 6 and user 1: 3
Predicted Rating for movie: 13 and user 1: 3
Predicted Rating for movie: 16 and user 1: 5
Predicted Rating for movie: 18 and user 1: 3
Predicted Rating for movie: 33 and user 1: 3
Predicted Rating for movie: 66 and user 1: 4
Predicted Rating for movie: 73 and user 1: 4
Predicted Rating for movie: 77 and user 1: 5
Predicted Rating for movie: 87 and user 1: 4
Predicted Rating for movie: 88 and user 1: 5
Predicted Rating for movie: 93 and user 1: 3
Predicted Rating for movie: 95 and user 1: 5


## 5. Item-based collaborative filtering algorithm
Item-based collaborative filtering methods presume that if fans of one item X are usually also fans of item Y and a user rates item X highly, he will probably like item Y as well, thus you should recommend it. 
The algorithm calculates the similarity (in this case cosine similarity) between the columns of the userItemRatings, i.e. the rating patterns of the items. 


To predict a rating of a movie a user has not yet rated, the our algorithm will calculate the find the top k, already rated items most similar to the given item, weights their rating by their similarity and calculates the mean to predict a rating for the user and this item. 

These methods often suffer from the cold-start problem, i.e. the rating userItemRating datasets having a lot of NAs. 

In [31]:
# calculate the item-based similarities between all items 
# transpose the matrix before to allow for column-wise calcuations
ratingItemSimilarities = cosine_similarity(userItemRatings.transpose())
ratingItemSimilarities

array([[1.        , 0.37468749, 0.26564983, ..., 0.20736762, 0.24062711,
        0.11841785],
       [0.37468749, 1.        , 0.23865713, ..., 0.41031879, 0.16002593,
        0.        ],
       [0.26564983, 0.23865713, 1.        , ..., 0.11425898, 0.06637753,
        0.        ],
       ...,
       [0.20736762, 0.41031879, 0.11425898, ..., 1.        , 0.33232227,
        0.        ],
       [0.24062711, 0.16002593, 0.06637753, ..., 0.33232227, 1.        ,
        0.        ],
       [0.11841785, 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [32]:
%%time
# function for getting the ids of the movies most similar to the active one
def get_most_similar_items(itemId, similarities, k=20, valid_ids=None):
    """
    Returns the top k items most similar to the given itemId
    itemId: the item id in question
    similarities: the matrix with the similarity between all columns of the userItemRatings matrix
    k: how many items' will be considered in finding similar items
    valid_ids: with this argument you can limit the ids you want to regard
    ... set this to be the ids of items that the user rated, 
    ... so that no similarity will be calculated for items not rated by the suer
    """
    # get the similarities of that particular movie
    similarities = similarities[itemId]
    # get the indices of the movie_ids ordered by similarity 
    most_similar_movie_ids = similarities.argsort()
    # reorder them from high to low
    most_similar_movie_ids = most_similar_movie_ids[::-1]
    # filter out any ids that are not listed in the array valid_ids
    if valid_ids is not None:
        most_similar_movie_ids = most_similar_movie_ids[np.isin(most_similar_movie_ids, valid_ids)]
    # the first one is the movie itself, you can ditch that one
    # return the k most similar movie_ids from the remaining ones and their similarities
    return most_similar_movie_ids[1:k], similarities[most_similar_movie_ids[1:k]]

print(get_most_similar_items(99, ratingItemSimilarities))

(array([135,  93, 143,  22,   8, 139, 100,  42, 124,  30, 120,   6,  10,
        77,  18,   2,   4, 138,  36]), array([0.74535599, 0.60999428, 0.56980288, 0.55215763, 0.51298918,
       0.46188022, 0.38313051, 0.38124643, 0.36363636, 0.33686077,
       0.31622777, 0.30323922, 0.29669541, 0.28261671, 0.27801922,
       0.26785981, 0.25048972, 0.23488809, 0.22610782]))
CPU times: user 2.43 ms, sys: 0 ns, total: 2.43 ms
Wall time: 2.02 ms


In [33]:
# predict the rating of a user and an unknown movie
def predict_rating_ibcf(userId, itemId, userItemRatings, similaritiesMatrix=None):
    """
    Item-based collaborative filtering
    userId, itemId: int, the user-id and item-id that you want to predict a rating for
    userItemRatings: matrix with users in rows, items in columns and ratings as values
    similaritiesMatrix: matrix with the similarity between every column in the userItemRatings
    """
    # this should not be done everytime the function is called
    # but in case someone doesnt compute the similarities beforehand, we do it here
    if similaritiesMatrix is None:
        # getting similarities between ratings 
        # transpose the matrix before to allow for column-wise calcuations
        similaritiesMatrix = cosine_similarity(userItemRatings.transpose())
        
    # get the sparse ratings for the user
    userRatings = userItemRatings[userId,:].todense()
    # get the indices of the items rated by him
    userRatingIds = userRatings.nonzero()
    # make get a matrix with the indices and the corresponding ratings
    userRating = np.array([
      userRatingIds[1],
      np.array(userRatings[userRatingIds])[0]
      ])
    # create an argsort array; this returns the indices to grab to get the array in order
    sortBy = np.argsort(userRating[1])
    sortBy = np.flip(sortBy)
    # recreate the 2d-array, but use the sorted indices
    userRating =userRating[0][sortBy]
    # we need integer indices for indexing later …
    userRating = userRating.astype(int)
    # userRating is now an array of the indices of the movies rated by the user, sorted by the rating
    
    # find the k most similar movies to the movie in question
    most_similar_movies, most_similar_movies_sims = get_most_similar_items(itemId, ratingItemSimilarities, valid_ids=userRating)
    # if no similar movies could be found, just return NaN
    if most_similar_movies.size == 0:
        return np.nan
    # calculate the relative similarities of the movies
    relative_similarities = most_similar_movies_sims / sum(most_similar_movies_sims)
    # calculate the weighted_ratings by multiplying the ratings of the top most similar movies by the relative similarities
    weighted_ratings = userItemRatings[userId, most_similar_movies] * relative_similarities[0]

    # calculate the average weighted rating
    average_weighted_rating = np.mean(weighted_ratings)
    # round up! but do not go higher than 5
    if not np.isnan(average_weighted_rating):
        average_weighted_rating = min(5, ceil(average_weighted_rating))
    return average_weighted_rating



#### Predicting movie ratings using item-based collaborative filtering


In [34]:
# Example
for userId in range(3):
    for itemId in range(100):
        try:
            rating = predict_rating_ibcf(userId, itemId, userItemRatings, ratingItemSimilarities)
            if rating > 1 or (userId == 0 and itemId < 2):
                print(f"Predicted Rating for movie: {itemId} and user {userId}:", rating)
        except:
            continue

Predicted Rating for movie: 0 and user 0: nan
Predicted Rating for movie: 1 and user 0: nan
Predicted Rating for movie: 35 and user 1: 2
Predicted Rating for movie: 40 and user 1: 2
Predicted Rating for movie: 48 and user 1: 2
Predicted Rating for movie: 72 and user 1: 2
Predicted Rating for movie: 74 and user 1: 3
Predicted Rating for movie: 80 and user 1: 2
Predicted Rating for movie: 90 and user 1: 2
Predicted Rating for movie: 98 and user 1: 4
Predicted Rating for movie: 0 and user 2: 3
Predicted Rating for movie: 1 and user 2: 3
Predicted Rating for movie: 2 and user 2: 4
Predicted Rating for movie: 3 and user 2: 4
Predicted Rating for movie: 4 and user 2: 4
Predicted Rating for movie: 5 and user 2: 3
Predicted Rating for movie: 6 and user 2: 4
Predicted Rating for movie: 7 and user 2: 4
Predicted Rating for movie: 8 and user 2: 4
Predicted Rating for movie: 9 and user 2: 3
Predicted Rating for movie: 10 and user 2: 3
Predicted Rating for movie: 11 and user 2: 3
Predicted Rating f

<ipython-input-33-6789343dd097>:40: RuntimeWarning: invalid value encountered in true_divide
  relative_similarities = most_similar_movies_sims / sum(most_similar_movies_sims)


Predicted Rating for movie: 20 and user 2: 3
Predicted Rating for movie: 21 and user 2: 3
Predicted Rating for movie: 22 and user 2: 3
Predicted Rating for movie: 23 and user 2: 4
Predicted Rating for movie: 24 and user 2: 3
Predicted Rating for movie: 25 and user 2: 3
Predicted Rating for movie: 27 and user 2: 4
Predicted Rating for movie: 28 and user 2: 3
Predicted Rating for movie: 30 and user 2: 3
Predicted Rating for movie: 31 and user 2: 3
Predicted Rating for movie: 32 and user 2: 3
Predicted Rating for movie: 34 and user 2: 3
Predicted Rating for movie: 36 and user 2: 3
Predicted Rating for movie: 38 and user 2: 3
Predicted Rating for movie: 39 and user 2: 4
Predicted Rating for movie: 40 and user 2: 4
Predicted Rating for movie: 41 and user 2: 3
Predicted Rating for movie: 42 and user 2: 3
Predicted Rating for movie: 44 and user 2: 3
Predicted Rating for movie: 45 and user 2: 4
Predicted Rating for movie: 46 and user 2: 3
Predicted Rating for movie: 47 and user 2: 3
Predicted 

## 6. Hybrid RecSys Algorithm

The authors of the referenced paper proposed a hybrid approach that works as follows:
1. Predict the rating of every unrated item using content-based filtering. The accuracy of these predicted ratings will likely be very low. These ratings will be called pseudo-ratings, as they will only aid the process of item-based collaborative filtering. 
2. Perform item-based collaborative filtering on the pseudo-ratings. This way the cold-start-problem will be sort of bypassed, because there will be fewer empty cells in the userItemRating matrix.

In [35]:
%%time
# create a copy of the user-item-ratings-matrix to fill in th pseudo ratings
def get_pseudoratings_cbf(userItemRatings, itemKeywordsMatrix):
    """
    Takes in a userItemRatings-matrix, fills the unrated items' cells with pseudo ratings and returns the matrix 
    userItemRatings: matrix with users in rows, items in columns and ratings as values
    itemKeywordsMatrix: matrix with items in rows, keywords in columns and matches as values
    """
    pseudoRatings = userItemRatings.copy()
    print("Entry file")
    #display(pseudoRatings)
    print("File Size:", sys.getsizeof(pseudoRatings))
    # replace every unrated user-movie field with a content-based predicted rating -> pseudo-rating!
    # iterate over all rows (users)
    for userId, row in enumerate(pseudoRatings):
        start_time = time.time()
        # select the user's row
        userRow = pseudoRatings.todense()[userId]
        userRow = np.array(userRow)[0]
        # get the ids of the rated and unrated movies
        unratedIds = userRow == 0
        # iterate through each film the user hasn't rated yet
        for unratedFilmId,_ in enumerate(userRow[unratedIds]):
            
            rating = predict_rating_cbf(userId, itemId, userItemRatings, itemKeywordsMatrix)
            
            # save the pseudoRating to the userRow 
            pseudoRatings[userId, unratedFilmId] = rating
            
        print("User", userId, "done –", round(time.time() - start_time, 3), "seconds taken", end="\r")
    print()
    print("Exit file")
    #display(pseudoRatings)
    print("File Size:", sys.getsizeof(pseudoRatings))
    return pseudoRatings

pseudoRatings = get_pseudoratings_cbf(userItemRatings, itemKeywordsMatrix) 
print("Filesize of the matrix:", sys.getsizeof(pseudoRatings))


Entry file
File Size: 48


<ipython-input-29-a44bce28ccea>:22: RuntimeWarning: invalid value encountered in true_divide
  sim = sim / sum(sim)
/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


User 181 done – 0.382 seconds taken
Exit file
File Size: 48
Filesize of the matrix: 48
CPU times: user 1min 10s, sys: 265 ms, total: 1min 10s
Wall time: 1min 10s


In [36]:
if itemLimit < 1000:
    p = get_sparse_pandas_df(pseudoRatings)
    display(p)

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,3,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
178,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
# function for making rating predictions of a user and a movie
def get_hybrid_recommendation(userId, itemId, userItemRatings, pseudoRatingsMatrix, ratingItemSimilarities):
    """
    Returns a recommendation using a hybrid approach (content-based filtering and item-based collaborative filtering)
    userId, itemId: int, the user-id and item-id that you want to predict a rating for
    pseudoRatingsMatrix: matrix with users in rows, items in columns and ratings as values, 
    ... including the content-based filtering predicted ratings
    userItemRatings: matrix with users in rows, items in columns and ratings as values
    ratingItemSimilarities: matrix with the similarity between every column in the userItemRatings
    """
    try:
        if userItemRatings[userId, itemId]:
            #print("Movie already rated")
            return userItemRatings[userId, itemId]
    except IndexError:
        #print("index error!", userId, itemId)
        return np.nan
    else:
        predictedRating = predict_rating_ibcf(userId, itemId, 
                                                 userItemRatings=pseudoRatingsMatrix, 
                                                 similaritiesMatrix=ratingItemSimilarities
                                                )
        return predictedRating

# Example
print("Hybrid Recommendation")
#print("item limit", itemLimit)
userId = 1
for i in range(30,40):
    itemId = i
    rec = get_hybrid_recommendation(userId, itemId, userItemRatings, pseudoRatings, ratingItemSimilarities)    
    print(f"Rating for user {userId} and movie {itemId} predicted:", rec)

Hybrid Recommendation
Rating for user 1 and movie 30 predicted: 3
Rating for user 1 and movie 31 predicted: 3
Rating for user 1 and movie 32 predicted: 3
Rating for user 1 and movie 33 predicted: nan
Rating for user 1 and movie 34 predicted: 3
Rating for user 1 and movie 35 predicted: nan
Rating for user 1 and movie 36 predicted: 5
Rating for user 1 and movie 37 predicted: 3
Rating for user 1 and movie 38 predicted: nan
Rating for user 1 and movie 39 predicted: nan


<ipython-input-33-6789343dd097>:40: RuntimeWarning: invalid value encountered in true_divide
  relative_similarities = most_similar_movies_sims / sum(most_similar_movies_sims)


## 7. Testing
To test the hybrid algorithm, we predict some already known ratings and get a confusion matrix.

In [39]:
%%time
# create a copy of the ratings and predict a rating for every rated movie
# then we can compare it with the actual ratings and calculate accuracy and other metrics

def get_test_ratings_hybrid(userItemRatings, pseudoRatings, ratingItemSimilarities):
    """
    Predicts the rating of all rated items in the userItemRatings matrix to allow for accuracy testing
    userItemRatings: matrix with users in rows, items in columns and ratings as values
    pseudoRatingsMatrix: matrix with users in rows, items in columns and ratings as values, 
    ... including the content-based filtering predicted ratings
    ratingItemSimilarities: matrix with the similarity between every column in the userItemRatings
    """
    userItemRatingsTest = userItemRatings.copy()
    # iterate through all users
    for userId,row in enumerate(userItemRatingsTest):
        # update message
        if userId % 500 == 0 and userId: 
            print(f"{i} tested")
        # make the row an accessible array
        row = np.array(row.todense())[0]
        # get the ids where the ratings are known
        ratedIds = np.where(row != 0)[0]
        # overwrite the ratings to be 0
        userItemRatingsTest[ratedIds] = 0
        # now predict the rating for every previously known rating
        for itemId in ratedIds:
            try:
                userItemRatingsTest[userId, itemId] = get_hybrid_recommendation(userId, itemId, userItemRatings, pseudoRatings, ratingItemSimilarities)
            except ValueError as e:
                pass
                #print("Value error", userId, itemId)
                #print(e)
            #print(userItemRatingsTest[userId, itemId])
    return userItemRatingsTest

userItemRatingsTest = get_test_ratings_hybrid(userItemRatings, pseudoRatings, ratingItemSimilarities)
print(userItemRatingsTest.todense())
print(sys.getsizeof(userItemRatingsTest))
print(userItemRatingsTest.shape)
print(userItemRatings.shape)
userItemRatingsTest

/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:124: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)
/opt/conda/lib/python3.8/site-packages/scipy/sparse/_index.py:82: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
48
(182, 149)
(182, 149)
CPU times: user 224 ms, sys: 4.07 ms, total: 228 ms
Wall time: 221 ms


<182x149 sparse matrix of type '<class 'numpy.int64'>'
	with 11791 stored elements in Compressed Sparse Row format>

In [40]:
if itemLimit < 1000:
    t = get_sparse_pandas_df(userItemRatingsTest)
    display(t)

,0,1,2,3,4,5,6,7,8,9,...,139,140,141,142,143,144,145,146,147,148
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,4,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177,0,0,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0
178,0,0,0,0,4,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
180,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## 8. Evaluation
To test the hybrid algorithm, we predicted some already known ratings. Now we can measure the accuracy for the ratings, and for the likes we measured accuracy, precision and recall.

In [41]:
# create functions to extract accuracy, precision and recall from matrices of predictions

def total(x):
    return sum(sum(x))

def true_positives(x,y): # x:actual, y: predicted
    tp = np.logical_and(x.copy(), y.copy())
    return tp

def true_negatives(x,y): # x:actual, y: predicted
    pn = np.logical_not(y.copy())
    nn = np.logical_not(x.copy(), y.copy())
    tn = np.logical_and(pn, nn)
    return tn

def false_positives(x,y): # x:actual, y: predicted
    n = np.logical_not(x.copy())
    fp = np.logical_and(y.copy(), n)
    return fp

def false_negatives(x,y): # x:actual, y: predicted
    pn = np.logical_not(y.copy())
    fn = np.logical_and(pn, x.copy())
    return fn

def accuracy(x,y): # x:actual, y: predicted
    return (total(true_positives(x,y)) + total(true_negatives(x,y))) / (total(x == x))

def precision(x,y): # x:actual, y: predicted
    return total(true_positives(x,y)) / (total(true_positives(x,y)) + total(false_positives(x,y)))

def recall(x,y): # x:actual, y: predicted
    return total(true_positives(x,y)) / (total(true_positives(x,y)) + total(false_negatives(x,y)))

In [42]:
# convert the ratings and predicted ratings to binary (like, not like)
likes = userItemRatings.copy() >= 3
likes = np.array(likes.todense())

likes_test = userItemRatingsTest.copy() >= 3
likes_test = np.array(likes_test.todense())


likes_pseudo = pseudoRatings.copy() >= 3
likes_pseudo = np.array(likes_pseudo.todense())


In [43]:
# Testing the pseudo-recommendations
print("Accuracy:", accuracy(likes, likes_pseudo))
print("Precision:", precision(likes, likes_pseudo))
print("Recall:", recall(likes, likes_pseudo))

Accuracy: 0.9461980972048086
Precision: 0.6518518518518519
Recall: 0.11421155094094744


In [44]:
# Testing the hybrid-recommendations
print("Accuracy:", accuracy(likes, likes_test))
print("Precision:", precision(likes, likes_test))
print("Recall:", recall(likes, likes_test))

Accuracy: 0.9788332472896232
Precision: 0.9958974358974358
Recall: 0.6301103179753407


### Discussion
The hybrid model performs with a relatively high precision but a quite low recall. 

A **high precision** means the recommendation system will make very few bad movie suggestions, i.e. it seldom rated movies highly that the respective user would have rated poorly. 
On the other hand, **low recall** means that the model will rate a lot of movies poorly when the respective user may have rated it more highly. 




## 9. References

Poonam Sharma, Lokesh Yadav (2020). *Movie Recommendation System Using Item Based Collaborative Filtering.* International Journal of Innovative Research in Computer Science & Technology (IJIRCST) ISSN: 2347-5552, Volume-8, Issue-4, July 2020 https://doi.org/10.21276/ijircst.2020.8.4.2